Here we should give the model some docs, then let it find out the topics

In [20]:
!pip install openai langchain 


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\Mahinour Elsarky\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [19]:
# LangChain basics
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain



# Langchain Loaders:
from langchain.document_loaders import YoutubeLoader

# Vector Store and retrievals
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
#import pinecone

# Chat Prompt templates for dynamic values
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# Supporting libraries
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:

llm3 = ChatOpenAI(temperature=0,
                  model_name="gpt-3.5-turbo-0613",
                  request_timeout = 180
                )

#llm3= OpenAI(model_name="gpt-3.5-turbo-0613", )

In [4]:
!pip install youtube-transcript-api


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\Mahinour Elsarky\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [5]:
!pip install tiktoken


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\Mahinour Elsarky\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [6]:
youtube_loader = YoutubeLoader.from_youtube_url("https://youtu.be/5p248yoa3oE?si=TATgA2GMtcQ_MjEA")
transcript = youtube_loader.load()

text_splitter = RecursiveCharacterTextSplitter(separators=["\n", " "], chunk_size=10000, chunk_overlap=2200)
docs = text_splitter.split_documents(transcript)
print (f"You have {len(docs)} docs. First doc is {llm3.get_num_tokens(docs[0].page_content)} tokens")



You have 5 docs. First doc is 1982 tokens


In [7]:

# % START OF EXAMPLES
# - Sam's Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
# - Shaan's Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
# - Revenge Against The Spam Calls: A couple of businesses focused on protecting consumers: RoboCall, TrueCaller, DoNotPay, FitIt
# - Wildcard CEOs vs. Prudent CEOs: However, Munger likes to surround himself with prudent CEO's and says he would never hire Musk.
# - Chess Business: Priyav, a college student, expressed his doubts on the MFM Facebook group about his Chess training business, mychesstutor.com, making $12.5K MRR with 90 enrolled.
# - Restaurant Refiller: An MFM Facebook group member commented on how they pay AirMark $1,000/month for toilet paper and toilet cover refills for their restaurant. Shaan sees an opportunity here for anyone wanting to compete against AirMark.
# - Collecting: Shaan shared an idea to build a mobile only marketplace for a collectors' category; similar to what StockX does for premium sneakers.
# % END OF EXAMPLES

template="""
You are a helpful assistant that helps retrieve distinct topics talked about in a youtube video transcript
- Your goal is to extract the topic names and brief 1-sentence description of the topic
- Topics can include:
- AI tools
- GPT Models
- Google Models
- LLMs
- llama Models
- Falcon Models
- Programming Languages
- AI recent News
- AI tutorials
- OpenAI
- AI for business 
- AI for education
- AI for medicine 
- AI for art and music
- Deep Learning
- NLP
- Machine Learning
- Data science
- Opportunities in AI
- AI frameworks
- Future AI
- Langchain

- Provide a brief description of the topics after the topic name. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the youtube video
- Do not respond with numbers, just bullet points of all topics listed under each other. Example:
Topics:
    - Topic 1 title: topic 1 description
    - Topic 2 title: topic 2 description
    - Topic 3 title: topic 3 description
    
- Ignore topics on policy and regulations
- Do not respond with anything outside of the transcript. If you can't extract any topics at all in the whole transcript, say 'Sorry, No topics found in the given content'
- Only pull topics from the transcript. Do not use the examples
- If the speakers names were mentioned in the transcript, instead of saying 'The speaker' refer to the names.
- Make your titles descriptive but concise. Example: 'Shaan's Experience at Twitch' should be 'Shaan's Interesting Projects At Twitch'
- A topic should be substantial, more than just a one-off comment

"""
#- Do not respond with anything outside of the transcript. If you don't see any topics, say, 'No Topics'
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Transcript: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_map = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])

In [8]:
# % START OF EXAMPLES
# - Sam's Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
# - Shaan's Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
# % END OF EXAMPLES

template="""
You are a helpful assistant that helps retrieve topics talked about in a youtube transcript
- You will be given a series of bullet topics of topics found
- Your goal is to exract the topic names and brief 1-sentence description of the topic
- Do not respond with numbers, just bullet points of all topics listed under each other.
- Deduplicate any bullet points you see
- If you think two or more topics are similar and can be merged, merge them together with one topic title and create a new description that fits the merged topics
- Only pull topics from the transcript. Do not use the examples.
"""
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Transcript: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_combine = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])



In [9]:
chain = load_summarize_chain(llm3,
                             chain_type="map_reduce",
                             map_prompt=chat_prompt_map,
                             combine_prompt=chat_prompt_combine,
                            verbose=True
                            )

In [10]:
topics_found = chain.run({"input_documents": docs})



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant that helps retrieve distinct topics talked about in a youtube video transcript
- Your goal is to extract the topic names and brief 1-sentence description of the topic
- Topics can include:
- AI tools
- GPT Models
- Google Models
- LLMs
- llama Models
- Falcon Models
- Programming Languages
- AI recent News
- AI tutorials
- OpenAI
- AI for business 
- AI for education
- AI for medicine 
- AI for art and music
- Deep Learning
- NLP
- Machine Learning
- Data science
- Opportunities in AI
- AI frameworks
- Future AI
- Langchain

- Provide a brief description of the topics after the topic name. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the youtube video
- Do not respond with numbers, just bullet points of all topics listed under each other. Example:
Topics:
    - Topic 1 title: topic 1 description
  

In [11]:
print(topics_found)

Topics:
- Opportunities in AI: Discussing the opportunities in AI and comparing it to electricity as a general-purpose technology.
- AI Tools: Mentioning supervised learning and generative AI as important tools in AI.
- Supervised Learning: Explaining how supervised learning is used for labeling things or computing input to outputs.
- Generative AI: Talking about the exciting tool of generative AI and mentioning GPT models.
- Large-scale Supervised Learning: Discussing the progress made in AI over the last decade through the use of large AI models and large amounts of data.
- Large Language Models: Explaining how large language models like GPT are built using supervised learning to predict the next word.
- Power of Large Language Models: Highlighting the power of large language models not only as consumer tools but also as developer tools, enabling faster development of AI applications.
- Workflow for Building AI Systems: Comparing the traditional workflow for building AI systems using

In [12]:
!pip install kor


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\Mahinour Elsarky\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [13]:

from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number


In [14]:
schema = Object(
    id="topic",
    description="Topic Information",
    examples=[
        ("Generative AI: An exciting tool in AI that can generate text based on input prompts.", [{"topic title": "Generative AI"}, {"description": "An exciting tool in AI that can generate text based on input prompts."}, {"tag": "AI Tools"}]),
        ("AI in relationships: The use of AI in relationship coaching and mentoring, such as the example of an AI-powered romantic relationship coaching app.", [{"topic title": "AI in relationships"}, {"description": "The use of AI in relationship coaching and mentoring, such as the example of an AI-powered romantic relationship coaching app."}, {"tag": "AI Applications"}]),
        ("AI in different industries: Exploring the potential of AI in various industries beyond consumer software and the internet.", [{"topic title": "AI in different industries"}, {"description": "Exploring the potential of AI in various industries beyond consumer software and the internet."}, {"tag": "AI Opportunities"}]),
        ("Supervised Learning: A technique in AI that is good at labeling things or computing input to outputs.", [{"topic title": "Supervised Learning"}, {"description": "A technique in AI that is good at labeling things or computing input to outputs."}, {"tag": "AI Tools"}]),
        ("Large Language Models: The power and potential of large language models in AI applications.", [{"topic title": "Large Language Models"}, {"description": "The power and potential of large language models in AI applications."}, {"tag": "AI LLMs"}]),
        ("Future growth of AI technologies: The prediction that supervised learning and generative AI will continue to grow in value and adoption over the next three years, with the potential for even greater expansion in the long term.", [{"topic title": "Future growth of AI technologies:"}, {"description": "The prediction that supervised learning and generative AI will continue to grow in value and adoption over the next three years, with the potential for even greater expansion in the long term."}, {"tag": "Future of AI"}]),
        ("Large Language Models: Dr. Andrew explains how large language models, like GPT, are built using supervised learning to predict the next word, enabling applications that can be built faster and more efficiently.", [{"topic title": "Large Language Models:"}, {"description": " Dr. Andrew explains how large language models, like GPT, are built using supervised learning to predict the next word, enabling applications that can be built faster and more efficiently."}, {"tag": "AI LLMs"}]),
 
    ],
    attributes=[
         Text(
            id="title",
            description="The title of the topic listed",
        ),
        Text(
            id="description",
            description="The description of the topic listed",
        ),
        Text(
            id="tag",
            description="The type of content being described",
        )
    ],
    many=True,
)

In [15]:
#chain = create_extraction_chain(schema, llm3)
chain = create_extraction_chain(llm3, schema)

In [16]:
topics_structured = chain.run(topics_found)

In [17]:
topics_structured

{'data': {'topic': [{'title': 'Opportunities in AI',
    'description': 'Discussing the opportunities in AI and comparing it to electricity as a general-purpose technology.',
    'tag': 'AI Opportunities'},
   {'title': 'AI Tools',
    'description': 'Mentioning supervised learning and generative AI as important tools in AI.',
    'tag': 'AI Tools'},
   {'title': 'Supervised Learning',
    'description': 'Explaining how supervised learning is used for labeling things or computing input to outputs.',
    'tag': 'AI Tools'},
   {'title': 'Generative AI',
    'description': 'Talking about the exciting tool of generative AI and mentioning GPT models.',
    'tag': 'AI Tools'},
   {'title': 'Large-scale Supervised Learning',
    'description': 'Discussing the progress made in AI over the last decade through the use of large AI models and large amounts of data.',
    'tag': 'AI Tools'},
   {'title': 'Large Language Models',
    'description': 'Explaining how large language models like GPT are

Now we will try with a website
